# Fierce Research ATR Position Sizing For Stocks

#### Import Packages

In [1]:
import numpy as np
import pandas as pd

import warnings 
warnings.filterwarnings('ignore')

#### Import Data

In [2]:
import eikon as tr
tr.set_app_key('DEFAULT_CODE_BOOK_APP_KEY')

stock = 'GNW'

df1, e= tr.get_data([stock],['tr.priceclose.date','tr.priceclose','tr.open','tr.high','tr.low'],
                       {'SDate':-50,'EDate':0,'Frq':'D'})

df1['Date'] = pd.to_datetime(df1['Date'])
df1.set_index('Date', inplace=True) 
df1.index = df1.index.map(lambda t: t.strftime('%Y-%m-%d'))
df1.columns = map(str.lower, df1.columns)
df1 = df1.drop_duplicates()
df1 = df1.round(2)

#### True Range

The TR for a given trading period is the greatest of the following:

- Current high minus the previous close
- Current low minus the previous close
- Current high minus the current low

In [3]:
df1['price close shifted'] = df1['price close'].shift(1)
df1['high minus low'] = df1['price high']-df1['price low']
df1['low minus previous close'] = df1['price low']-df1['price close shifted']
df1['high minus previous close'] = df1['price high']-df1['price close shifted']
df1['true range'] = abs(df1[['high minus low','low minus previous close','high minus previous close']]).max(axis=1)
df1.tail()

,instrument,price close,price open,price high,price low,price close shifted,high minus low,low minus previous close,high minus previous close,true range
Date,,,,,,,,,,
2021-02-23,GNW,3.03,3.03,3.16,2.95,3.16,0.21,-0.21,0.00,0.21
2021-02-24,GNW,3.13,3.11,3.15,3.04,3.03,0.11,0.01,0.12,0.12
2021-02-25,GNW,3.18,3.14,3.29,3.05,3.13,0.24,-0.08,0.16,0.24
2021-02-26,GNW,3.12,3.19,3.26,3.11,3.18,0.15,-0.07,0.08,0.15
2021-03-01,GNW,3.31,3.25,3.31,3.17,3.12,0.14,0.05,0.19,0.19


#### Average True Range

In [4]:
df1['atr'] = (df1['high minus low'].rolling(13).sum()+df1['high minus low'])/14

In [5]:
df1['2x atr'] = df1['atr']*2
df1.tail()

,instrument,price close,price open,price high,price low,price close shifted,high minus low,low minus previous close,high minus previous close,true range,atr,2x atr
Date,,,,,,,,,,,,
2021-02-23,GNW,3.03,3.03,3.16,2.95,3.16,0.21,-0.21,0.00,0.21,0.307857,0.615714
2021-02-24,GNW,3.13,3.11,3.15,3.04,3.03,0.11,0.01,0.12,0.12,0.271429,0.542857
2021-02-25,GNW,3.18,3.14,3.29,3.05,3.13,0.24,-0.08,0.16,0.24,0.279286,0.558571
2021-02-26,GNW,3.12,3.19,3.26,3.11,3.18,0.15,-0.07,0.08,0.15,0.263571,0.527143
2021-03-01,GNW,3.31,3.25,3.31,3.17,3.12,0.14,0.05,0.19,0.19,0.258571,0.517143


#### % Of Capital At Risk

In [6]:
df1['$ at risk'] = (.002*110000)

In [7]:
df1['recommended position size'] = df1['$ at risk']/df1['2x atr']

#### Go To Market Position Size

In [8]:
df1['recommended position size'][-1].round()

425.0